<a href="https://colab.research.google.com/github/matdjohnson-at-umass-dot-edu/cs646-final-project/blob/main/CS646_Final_Project_Classifier_Evaluator_Continued_Effort_Instance2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets
! pip install transformers
! pip install pytrec_eval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=308204 sha256=6e38379afa5dd3

In [2]:
from datasets import concatenate_datasets, Dataset, disable_caching, disable_progress_bars, load_dataset
from tqdm import tqdm
from google.colab import drive
import os
import torch
import torch.nn.functional as torch_func
import gc
import time
from threading import Lock
from concurrent.futures import ThreadPoolExecutor
from transformers import AutoTokenizer, AutoModel
import logging
import psutil
import numpy as np
from collections import Counter
import random
import math
import psutil
import pytrec_eval

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class SimpleAttentionModel_1AttnModule(torch.nn.Module):
    def __init__(self, emb_dim=1024, inner_dim=2048, num_attn_heads=1, linear_layer_dim=8196):
        super().__init__()
        self.query_proj = torch.nn.Linear(in_features=emb_dim*2, out_features=inner_dim)
        self.key_proj = torch.nn.Linear(in_features=emb_dim*2, out_features=inner_dim)
        self.value_proj = torch.nn.Linear(in_features=emb_dim*2, out_features=inner_dim)
        self.layer_1 = torch.nn.MultiheadAttention(embed_dim=inner_dim, num_heads=num_attn_heads, batch_first=True)
        self.layer_2 = torch.nn.ReLU()
        self.layer_3 = torch.nn.Linear(in_features=inner_dim, out_features=linear_layer_dim)
        self.layer_4 = torch.nn.Linear(in_features=linear_layer_dim, out_features=2)
        self.layer_5 = torch.nn.LogSoftmax(dim=1)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(device=device)

    def forward(self, query_embs, doc_embs):
        assert len(query_embs.shape) == 2 and len(doc_embs.shape) == 2 and query_embs.shape[0] == doc_embs.shape[0] and query_embs.shape[1] == doc_embs.shape[1]
        concatenated_embs = torch.concatenate(tensors=[query_embs, doc_embs], axis=-1)
        query = self.query_proj(concatenated_embs).unsqueeze(-2)
        key = self.key_proj(concatenated_embs).unsqueeze(-2)
        value = self.value_proj(concatenated_embs).unsqueeze(-2)
        layer_1_output = self.layer_1(
            query,
            key,
            value
        )
        layer_1_output_squeezed = layer_1_output[0].squeeze(-2)
        layer_2_output = self.layer_2(layer_1_output_squeezed)
        layer_3_output = self.layer_3(layer_2_output)
        layer_4_output = self.layer_4(layer_3_output)
        layer_5_output = self.layer_5(layer_4_output)
        return layer_5_output

In [4]:
models_base_dir = "/content/drive/MyDrive/CS646-FinalProject/models"

hyper_parameters_2024_12_14_03 = {
    "epochs": 10,
    "max_lr": 0.00025,
    "fraction_of_epochs_as_warmup": 0.1,
    "fraction_of_max_lr_at_init": 0.8,
    "fraction_of_max_lr_at_end": 0.33,
    "batch_size": 700,
    "num_attn_modules": 1,
    "num_attn_heads": 4,
    "linear_layer_dim": 8196,
    "parameter_set_name": "hyper_parameters_2024_12_14_03"
}

hyper_parameters = hyper_parameters_2024_12_14_03

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

timestamp = "20241215T170255"

model_parameter_file = f"{models_base_dir}/{hyper_parameters['parameter_set_name']}-{timestamp}.pt"

model = torch.load(model_parameter_file, map_location=device)

<ipython-input-4-818b20abed16>:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_parameter_file, map_location=device)


In [5]:
datasets_base_dir = "/content/drive/MyDrive/CS646-FinalProject/datasets"
final_dataset_train_file_path_and_name = f"{datasets_base_dir}/ms_marco_final_dataset_avg/ms_marco_final_dataset_avg_train.parquet"
final_dataset_test_file_path_and_name = f"{datasets_base_dir}/ms_marco_final_dataset_avg/ms_marco_final_dataset_avg_test.parquet"
final_dataset_train = Dataset.from_parquet(final_dataset_train_file_path_and_name)
final_dataset_test = Dataset.from_parquet(final_dataset_test_file_path_and_name)
final_dataset = concatenate_datasets([final_dataset_train, final_dataset_test])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

final_dataset = final_dataset.with_format("torch", device=device)

doc_ids = torch.concatenate((
    final_dataset['pos_doc_id'],
    final_dataset['neg_doc_id']
))
doc_embs = torch.concatenate((
    final_dataset['pos_doc_emb'],
    final_dataset['neg_doc_emb']
))

doc_ids = doc_ids.detach().cpu()

final_dataset_test = final_dataset_test.with_format("torch", device=device)

query_ids = final_dataset_test['query_id'].tolist()
query_embs = final_dataset_test['query_emb']

del final_dataset_train, final_dataset_test, final_dataset
gc.collect()
torch.cuda.empty_cache()
gc.collect()

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

0

In [6]:
print(f"doc_ids.shape:{doc_ids.shape} doc_ids:{doc_ids}")
print(f"doc_embs.shape:{doc_embs.shape} doc_embs:{doc_embs}")
print(f"len(query_ids):{len(query_ids)} query_ids:{query_ids}")
print(f"query_embs.shape:{query_embs.shape} query_embs:{query_embs}")

doc_ids.shape:torch.Size([423992]) doc_ids:tensor([ 254896, 6791745, 2057320,  ...,  112417,  148897,  151981])
doc_embs.shape:torch.Size([423992, 1024]) doc_embs:tensor([[-0.1530,  0.1937, -0.2566,  ..., -2.3070,  0.4499, -0.5741],
        [ 0.5722,  0.5817, -0.2694,  ...,  0.6706, -0.3559,  1.0092],
        [-0.9252, -0.2298, -1.8285,  ...,  0.9949, -0.5199,  0.8378],
        ...,
        [-0.1360, -0.2837, -0.3161,  ...,  0.4891,  0.4375, -0.0152],
        [ 0.8175,  0.7817, -0.5155,  ..., -0.4142,  0.2188, -0.0254],
        [-0.5934,  0.3350,  0.1335,  ..., -0.3068,  0.4187, -0.0288]],
       device='cuda:0')
len(query_ids):10600 query_ids:[704021, 956750, 279559, 860694, 388025, 863699, 56204, 443658, 271018, 396460, 1021600, 482191, 292541, 940542, 642276, 230072, 551066, 741761, 865760, 839891, 517145, 1004324, 26822, 257280, 1008608, 719589, 856479, 881234, 426899, 413977, 1038281, 1038442, 609395, 395242, 563193, 1007872, 538315, 734752, 625175, 287123, 260131, 936327, 183568,

In [7]:
log_freq = 100
batch_size = doc_embs.shape[0] // 2

results = {}

model.eval()

for i in range(0, query_embs.shape[0]):
    batches = doc_embs.shape[0] // batch_size
    prbs = None
    if not query_embs.shape[1] % batch_size == 0:
        batches = batches + 1
    inner_log_freq = max((batches // 10), 1)
    for j in range(0, batches):
        batch_lower_bound = j * batch_size
        if j < batches - 1:
            batch_upper_bound = (j + 1) * batch_size
        else:
            batch_upper_bound = doc_embs.shape[0]
        doc_embs_batch = doc_embs[batch_lower_bound:batch_upper_bound,:]
        query_rep_ct = batch_upper_bound - batch_lower_bound
        query_emb_rep = query_embs[i,:].repeat(query_rep_ct, 1)
        logits = model.forward(query_emb_rep, doc_embs_batch).detach().cpu()
        output_prbs = torch.tensor(torch.exp(logits), device="cpu")[:,0]
        if prbs is None:
            prbs = output_prbs
        else:
            prbs = torch.concatenate([prbs, output_prbs], axis=0)
        del logits, doc_embs_batch
        gc.collect()
        torch.cuda.empty_cache()
        gc.collect()
        if (i == 0 or i % log_freq == 0) and j % inner_log_freq == 0:
            timestamp = time.strftime("%Y-%m-%dT%H:%M:%S", time.localtime())
            print(f"{timestamp} query:{i}/{query_embs.shape[0]} batch:{j}/{batches} memory:{(torch.cuda.mem_get_info()[1] - torch.cuda.mem_get_info()[0]) / 1024 / 1024 / 1024}")
    k=1000
    top_k = torch.topk(
        prbs,
        k,
        dim=0
    )
    top_k_doc_ids = doc_ids.take(top_k[1]).detach().cpu().tolist()
    top_k_doc_scores = top_k[0].detach().cpu().tolist()
    result = dict(zip(
        map(lambda element: str(element), top_k_doc_ids),
        top_k_doc_scores
    ))
    if i == 0 or i % log_freq == 0:
        timestamp = time.strftime("%Y-%m-%dT%H:%M:%S", time.localtime())
        print(f"{timestamp} iter:{i+1}/{query_embs.shape[0]} result[{str(query_ids[i])}]: {result}"[0:1500])
    results[str(query_ids[i])] = result
    del query_emb_rep, prbs, top_k
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()


<ipython-input-7-6730ddcfe5e5>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output_prbs = torch.tensor(torch.exp(logits), device="cpu")[:,0]


2024-12-17T06:33:15 query:0/10600 batch:0/3 memory:3.9326171875
2024-12-17T06:33:17 query:0/10600 batch:1/3 memory:3.9326171875
2024-12-17T06:33:17 query:0/10600 batch:2/3 memory:3.1220703125
2024-12-17T06:33:17 iter:1/10600 result[704021]: {'2660961': 1.0, '4437848': 1.0, '5204280': 1.0, '1824028': 1.0, '1696907': 1.0, '214140': 1.0, '6615274': 1.0, '4111353': 1.0, '3835353': 1.0, '685642': 1.0, '5870645': 1.0, '1981353': 1.0, '1397157': 1.0, '4991813': 1.0, '1916842': 1.0, '1116458': 1.0, '3222789': 1.0, '6056715': 1.0, '5211612': 1.0, '4904128': 1.0, '557256': 1.0, '1796958': 1.0, '1972453': 1.0, '6213579': 1.0, '307585': 1.0, '1706636': 1.0, '5791802': 1.0, '4967443': 1.0, '5936729': 1.0, '1807425': 1.0, '5872401': 1.0, '1696041': 1.0, '6323889': 1.0, '627829': 1.0, '155164': 1.0, '3623456': 1.0, '1140504': 1.0, '2064030': 1.0, '4903989': 1.0, '1323964': 1.0, '2925886': 1.0, '528240': 1.0, '5869680': 1.0, '1809283': 1.0, '4954022': 1.0, '4815244': 1.0, '289811': 1.0, '1964873': 1.0

In [8]:
qrels_dataset = load_dataset('BeIR/msmarco-qrels')
qrels = {}
for entry in qrels_dataset['train']:
    if qrels.get(str(entry['query-id'])) is None:
        qrels[str(entry['query-id'])] = { str(entry['corpus-id']): int(entry['score']) }
    else:
        qrels[str(entry['query-id'])]['corpus-id'] = int(entry['score'])
for entry in qrels_dataset['test']:
    if qrels.get(str(entry['query-id'])) is None:
        qrels[str(entry['query-id'])] = { str(entry['corpus-id']): int(entry['score']) }
    else:
        qrels[str(entry['query-id'])]['corpus-id'] = int(entry['score'])
for entry in qrels_dataset['validation']:
    if qrels.get(str(entry['query-id'])) is None:
        qrels[str(entry['query-id'])] = { str(entry['corpus-id']): int(entry['score']) }
    else:
        qrels[str(entry['query-id'])]['corpus-id'] = int(entry['score'])
for entry in qrels_dataset['test']:
    if qrels.get(str(entry['query-id'])) is None:
        qrels[str(entry['query-id'])] = { str(entry['corpus-id']): int(entry['score']) }
    else:
        qrels[str(entry['query-id'])]['corpus-id'] = int(entry['score'])

rel_evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'recall.10', 'recall.100', 'recall.1000'})

rel_evalution = rel_evaluator.evaluate(results)
print(f"rel_evalution: {rel_evalution}"[:1500])

recall_10_sum = 0
recall_100_sum = 0
recall_1000_sum = 0

for query_id in rel_evalution.keys():
    recall_10_sum = recall_10_sum + rel_evalution[query_id]['recall_10']
    recall_100_sum = recall_100_sum + rel_evalution[query_id]['recall_100']
    recall_1000_sum = recall_1000_sum + rel_evalution[query_id]['recall_1000']

average_recall_10 = recall_10_sum / len(rel_evalution.keys())
average_recall_100 = recall_100_sum / len(rel_evalution.keys())
average_recall_1000 = recall_1000_sum / len(rel_evalution.keys())

print(f"average_recall_10: {average_recall_10}, average_recall_100: {average_recall_100}, average_recall_1000: {average_recall_1000}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

train.tsv:   0%|          | 0.00/9.52M [00:00<?, ?B/s]

dev.tsv:   0%|          | 0.00/136k [00:00<?, ?B/s]

test.tsv:   0%|          | 0.00/169k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/532751 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7437 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9260 [00:00<?, ? examples/s]

rel_evalution: {'704021': {'recall_10': 0.0, 'recall_100': 0.0, 'recall_1000': 0.0}, '956750': {'recall_10': 0.0, 'recall_100': 0.0, 'recall_1000': 0.0}, '279559': {'recall_10': 0.0, 'recall_100': 0.0, 'recall_1000': 0.0}, '860694': {'recall_10': 0.0, 'recall_100': 0.0, 'recall_1000': 0.0}, '388025': {'recall_10': 0.0, 'recall_100': 0.0, 'recall_1000': 0.0}, '863699': {'recall_10': 0.0, 'recall_100': 0.0, 'recall_1000': 0.0}, '56204': {'recall_10': 0.0, 'recall_100': 0.0, 'recall_1000': 0.0}, '443658': {'recall_10': 0.0, 'recall_100': 0.0, 'recall_1000': 0.0}, '271018': {'recall_10': 0.0, 'recall_100': 0.0, 'recall_1000': 0.0}, '396460': {'recall_10': 0.0, 'recall_100': 0.0, 'recall_1000': 0.0}, '1021600': {'recall_10': 0.0, 'recall_100': 0.0, 'recall_1000': 0.0}, '482191': {'recall_10': 0.0, 'recall_100': 0.0, 'recall_1000': 0.0}, '292541': {'recall_10': 0.0, 'recall_100': 0.5, 'recall_1000': 0.5}, '940542': {'recall_10': 0.0, 'recall_100': 0.0, 'recall_1000': 0.0}, '642276': {'recall